In [107]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from nltk import sent_tokenize
from gensim.utils import simple_preprocess
import gensim

In [63]:
temp_df = pd.read_csv("IMDB Dataset.csv")

In [64]:
df = temp_df.iloc[:10000]

In [65]:
df.head

<bound method NDFrame.head of                                                  review sentiment
0     One of the other reviewers has mentioned that ...  positive
1     A wonderful little production. <br /><br />The...  positive
2     I thought this was a wonderful way to spend ti...  positive
3     Basically there's a family where a little boy ...  negative
4     Petter Mattei's "Love in the Time of Money" is...  positive
...                                                 ...       ...
9995  Fun, entertaining movie about WWII German spy ...  positive
9996  Give me a break. How can anyone say that this ...  negative
9997  This movie is a bad movie. But after watching ...  negative
9998  This is a movie that was probably made to ente...  negative
9999  Smashing film about film-making. Shows the int...  positive

[10000 rows x 2 columns]>

In [66]:
df.review[1]

'A wonderful little production. <br /><br />The filming technique is very unassuming- very old-time-BBC fashion and gives a comforting, and sometimes discomforting, sense of realism to the entire piece. <br /><br />The actors are extremely well chosen- Michael Sheen not only "has got all the polari" but he has all the voices down pat too! You can truly see the seamless editing guided by the references to Williams\' diary entries, not only is it well worth the watching but it is a terrificly written and performed piece. A masterful production about one of the great master\'s of comedy and his life. <br /><br />The realism really comes home with the little things: the fantasy of the guard which, rather than use the traditional \'dream\' techniques remains solid then disappears. It plays on our knowledge and our senses, particularly with the scenes concerning Orton and Halliwell and the sets (particularly of their flat with Halliwell\'s murals decorating every surface) are terribly well d

In [67]:
df ['sentiment'].value_counts()

sentiment
positive    5028
negative    4972
Name: count, dtype: int64

In [68]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [69]:
df.duplicated().sum()

17

In [70]:
df.drop_duplicates(inplace= True)

/tmp/ipykernel_21881/2207841240.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace= True)


In [71]:
df.duplicated().sum()

0

In [72]:
def remove_html_tag(s):
    cleaned_text = re.sub(re.compile('<*?>'), '', s)
    return cleaned_text

In [73]:
df['review'] = df['review'].apply(remove_html_tag)

/tmp/ipykernel_21881/1636111163.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_html_tag)


In [74]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /<br /The f...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
9995,"Fun, entertaining movie about WWII German spy ...",positive
9996,Give me a break. How can anyone say that this ...,negative
9997,This movie is a bad movie. But after watching ...,negative
9998,This is a movie that was probably made to ente...,negative


In [75]:
df['review'] = df['review'].apply(lambda x:x.lower())

/tmp/ipykernel_21881/740760900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [76]:
sw_list = stopwords.words('english')
df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

/tmp/ipykernel_21881/4110084704.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [77]:
df

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. <br /<br /the fil...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive
...,...,...
9995,"fun, entertaining movie wwii german spy (julie...",positive
9996,"give break. anyone say ""good hockey movie""? kn...",negative
9997,movie bad movie. watching endless series bad h...,negative
9998,"movie probably made entertain middle school, e...",negative


In [78]:
X = df.iloc[:,0:1]
y= df['sentiment']

In [79]:
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. <br /<br /the fil...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [80]:
y

0       positive
1       positive
2       positive
3       negative
4       positive
          ...   
9995    positive
9996    negative
9997    negative
9998    negative
9999    positive
Name: sentiment, Length: 9983, dtype: object

In [81]:
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [82]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [83]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=1)

In [84]:
X_train.shape

(7986, 1)

In [85]:
#Applying BOW
cv = CountVectorizer()

In [86]:
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()

In [87]:
X_train_bow.shape

(7986, 47918)

In [88]:
gnb = GaussianNB()
gnb.fit(X_train_bow, y_train)

GaussianNB()

In [89]:
y_pred = gnb.predict(X_test_bow)
accuracy_score(y_test, y_pred)

0.6349524286429644

In [90]:
confusion_matrix(y_test, y_pred)

array([[716, 236],
       [493, 552]])

In [91]:
rf = RandomForestClassifier()

rf.fit(X_train_bow, y_train)

RandomForestClassifier()

In [92]:
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test, y_pred)

0.8487731597396094

In [93]:
cv = CountVectorizer(max_features=3000)
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()
rf = RandomForestClassifier()
rf.fit(X_train_bow, y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test, y_pred)

0.842764146219329

In [94]:
cv = CountVectorizer(ngram_range=(1,2))
X_train_bow = cv.fit_transform(X_train['review']).toarray()
X_test_bow = cv.transform(X_test['review']).toarray()
rf = RandomForestClassifier()
rf.fit(X_train_bow, y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test, y_pred)

MemoryError: Unable to allocate 44.3 GiB for an array with shape (7986, 745271) and data type int64

In [95]:
# Using Tfidf
tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])

In [96]:
rf = RandomForestClassifier()
rf.fit(X_train_bow, y_train)
y_pred = rf.predict(X_test_bow)
accuracy_score(y_test, y_pred)

0.8392588883324987

In [97]:
#Word2Vec
story = []
for doc in df ['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

In [102]:
model = gensim.models.Word2Vec(
    window =10,
    min_count =2
)

In [103]:
model.build_vocab(story)

In [104]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)

(5936675, 6424695)

In [105]:
len(model.wv.index_to_key)

31846

In [108]:
def document_vector(doc):
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis =0)


In [109]:
document_vector(df['review'].values[0])

array([-2.25338444e-01,  3.27891767e-01,  9.84064117e-02, -1.23554673e-02,
        1.68802515e-01, -6.74279094e-01,  2.69629806e-01,  9.00903165e-01,
       -2.60156002e-02, -2.55652547e-01, -5.04387058e-02, -5.08259416e-01,
        1.71463862e-02,  3.80541146e-01,  6.51822686e-02, -2.23759055e-01,
        1.80319935e-01, -3.64586323e-01,  8.79114494e-02, -5.74481308e-01,
       -6.90196753e-02,  1.51211858e-01,  3.41127217e-01, -9.81510580e-02,
        8.10483843e-02, -5.02676852e-02, -4.19535488e-01, -1.35345981e-01,
       -4.70194072e-01,  4.42035757e-02,  2.45814428e-01,  4.46835096e-04,
        2.73035169e-01, -3.35072160e-01, -9.10470262e-02,  5.26168585e-01,
       -2.64136512e-02, -2.03649208e-01, -3.12887520e-01, -7.54638553e-01,
       -1.74089193e-01, -2.09707782e-01, -1.91380873e-01,  1.13151051e-01,
        4.87125635e-01, -1.91190809e-01, -2.39475101e-01, -8.66108984e-02,
        4.79025692e-02,  1.37104258e-01,  1.26675054e-01, -2.68993825e-01,
       -2.82370299e-01, -